In [1]:
import re
import urllib
import pdftotext
from os import walk
import PyPDF2 
import textract
import pandas as pd 
import fitz # PyMuPDF
import io
import pytesseract
from PIL import Image
import cv2
import numpy as np 
import plotly.express as px

In [2]:
mypath = "diarios"
f = []
names = []
for (dirpath, dirnames, filenames) in walk(mypath):
    [f.append(f"{dirpath}/{file}") for file in filenames]
    names.append(dirnames)
#for file in f:
 #   with open(file,'rb') as f:
  #      pdfReader = PyPDF2.PdfFileReader(f)
   #     pageObj = pdfReader.getPage(0)     

In [138]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)



In [144]:
custom_config = r'--psm 6'
# iterate over PDF pages
for file in f:
    pdf_file = fitz.open(file)
    text = ""
    for page_index in range(len(pdf_file)):
        # get the page itself
        page = pdf_file[page_index]
        image_list = page.getImageList()
        # printing number of images found in this page
        if image_list:
            print(f"[+] Page {page_index} in Gazette {file[8:18]} Found a total of {len(image_list)} images in page {page_index}")
        else:
            print("[!] No images found on page", page_index)
        for image_index, img in enumerate(image_list, start=1):
            # get the XREF of the image
            xref = img[0]
            # extract the image bytes
            base_image = pdf_file.extractImage(xref)
            image_bytes = base_image["image"]
            # get the image extension
            image_ext = base_image["ext"]
            # load it to PIL
            image = Image.open(io.BytesIO(image_bytes))

            # save it to local disk
            img = cv2.imdecode(np.frombuffer(image_bytes, np.uint8),-1)
            #ocv_test = opening(img)
            text += pytesseract.image_to_string(img, lang='por',config=custom_config)  # Extraindo o texto da imagem

            #image.safve(open(f"image{page_index+1}_{image_index}.{image_ext}", "wb"))
    file = open(f"outputs/{file[8:18]}.txt","a+")
    file.write(text)
    file.close()
        #print(text)

5 in Gazette 2020-10-05 Found a total of 2 images in page 75
[+] Page 76 in Gazette 2020-10-05 Found a total of 2 images in page 76
[+] Page 77 in Gazette 2020-10-05 Found a total of 2 images in page 77
[+] Page 78 in Gazette 2020-10-05 Found a total of 2 images in page 78
[+] Page 79 in Gazette 2020-10-05 Found a total of 2 images in page 79
[+] Page 80 in Gazette 2020-10-05 Found a total of 2 images in page 80
[+] Page 81 in Gazette 2020-10-05 Found a total of 2 images in page 81
[+] Page 82 in Gazette 2020-10-05 Found a total of 2 images in page 82
[+] Page 83 in Gazette 2020-10-05 Found a total of 2 images in page 83
[+] Page 84 in Gazette 2020-10-05 Found a total of 2 images in page 84
[+] Page 85 in Gazette 2020-10-05 Found a total of 2 images in page 85
[+] Page 86 in Gazette 2020-10-05 Found a total of 2 images in page 86
[+] Page 87 in Gazette 2020-10-05 Found a total of 2 images in page 87
[+] Page 88 in Gazette 2020-10-05 Found a total of 2 images in page 88
[+] Page 89 in G

In [3]:
def create_dataframe(dirpath,filename):
    print(f"{dirpath}{filename}")
    gazette = dict()
    with open(filename, 'r') as file:
        gazette['date'] = filename[8:-4] #pega só a data no nome do arquivo.
        gazette['content'] = re.sub('\s+', ' ', file.read())
    #print(gazette)
    return gazette
df_gazettes = pd.DataFrame()
for (dirpath, dirnames, filenames) in walk("outputs/"):
    df_gazettes = df_gazettes.append([create_dataframe(dirpath,f"{dirpath}{file}") for file in filenames])

outputs/outputs/2020-11-07.txt
outputs/outputs/2020-11-06.txt
outputs/outputs/2020-11-04.txt
outputs/outputs/2020-11-05.txt
outputs/outputs/2020-11-03.txt
outputs/outputs/2020-10-19.txt
outputs/outputs/2020-10-27.txt
outputs/outputs/2020-10-26.txt
outputs/outputs/2020-10-22.txt
outputs/outputs/2020-10-23.txt
outputs/outputs/2020-10-09.txt
outputs/outputs/2020-10-21.txt
outputs/outputs/2020-10-20.txt
outputs/outputs/2020-10-08.txt
outputs/outputs/2020-10-05.txt
outputs/outputs/2020-10-06.txt
outputs/outputs/2020-10-07.txt
outputs/outputs/2020-10-13.txt
outputs/outputs/2020-10-03.txt
outputs/outputs/2020-10-02.txt
outputs/outputs/2020-10-16.txt
outputs/outputs/2020-10-28.txt
outputs/outputs/2020-10-14.txt
outputs/outputs/2020-10-15.txt
outputs/outputs/2020-10-01.txt
outputs/outputs/2020-10-29.txt
outputs/outputs/2020-11-19.txt
outputs/outputs/2020-11-18.txt


# Classificação dos Textos

## 1 - Quais as palavras mais frequentes por diário oficial

In [4]:
import spacy
from collections import Counter
from spacy.lang.pt import Portuguese
from spacy.lang.pt.stop_words import STOP_WORDS

In [5]:
#criando objeto para nlp em pt
nlp = Portuguese()
STOP_WORDS.add('e')
STOP_WORDS.add('a')
STOP_WORDS.add('o')

In [6]:
#teste
text = df_gazettes['content'][4].lower()
doc = nlp(text)

In [7]:
tipo = "licitação pregão concorrência inexigibilidade leilão carta-convite lei orçamento decreto projeto concurso edital"
area = "educação saúde finanças esporte lazer segurança ambiente social mobilidade"
nlp_tipo = nlp(tipo)
nlp_area = nlp(area)

In [8]:
doc_tokens = [token for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
doc_texts = [token.text.lower() for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
tipo_tokens = [token for token in nlp_tipo if token.is_punct != True]
area_tokens = [token for token in nlp_area if token.is_punct != True]

In [9]:
def gera_token(gazette):
    doc = nlp(gazette)
    doc_token =  [token for token in doc if (token.text.lower() not in STOP_WORDS) and token.is_punct != True and token.like_num != True and token.is_alpha == True]
    return doc_token
def classificacao(token_diario,token_tipo,token_area):
    tags_tipo = []
    tags_area = []
    for word in token_diario:
        for tipo in token_tipo:
            if word.similarity(tipo) >=0.9:
                #print(f"{word} - {tipo} = {word.similarity(tipo)}")
                tags_tipo.append(tipo)
            #else:
            #    tags_tipo.append("Outros")
        for area in token_area:
            if word.similarity(area) >=0.9:
                #print(f"{word} - {tipo} = {word.similarity(tipo)}")
                tags_area.append(area)
            #else:
             #   tags_area.append("Outros")
    return (tags_tipo,tags_area)

tipo = "licitação pregão concorrência inexigibilidade leilão carta-convite lei orçamento decreto projeto concurso edital"
area = "educação saúde finanças esporte lazer segurança ambiente social mobilidade"
#nlp_tipo = nlp(tipo)
#nlp_area = nlp(area)
tipo_tokens = gera_token(tipo)
area_tokens = gera_token(area)

In [10]:
tipo_tags = []
area_tags = []
gazette_date = []
for idx,gazette in df_gazettes.iterrows():
    #df_gazettes['tipo'] = 
    tipo_tags.append(set(classificacao(gera_token(gazette['content'].lower()),tipo_tokens,area_tokens)[0]))
    area_tags.append(set(classificacao(gera_token(gazette['content'].lower()),area_tokens,area_tokens)[1]))
    gazette_date.append(gazette['date'])
    #df_gazettes['area'] = classificacao(gera_token(gazette['content']),tipo_tokens,area_tokens)[1]

In [11]:
columns = area.split()+tipo.split()

In [12]:
all_columns = df_gazettes.columns.tolist()+columns

In [13]:
df_onehot = pd.DataFrame()
for tipo,date,area in zip(tipo_tags,gazette_date,area_tags):
    df_onehot['date'] = date
    dt = dict.fromkeys(tipo,1)
    da = dict.fromkeys(area,1)
    da.update(dt)
    df_onehot = df_onehot.append(da,ignore_index=True)
    #df_onehot = df_onehot.append(,ignore_index=True)


In [20]:
resultado = pd.DataFrame(df_onehot.fillna(0).sum(),columns=['Total']).reset_index()
resultado['index'] = resultado['index'].astype('str')

In [21]:
px.bar(resultado,x='index',y='Total')